In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/marketing-strategy-personalised-offer/sample.csv
/kaggle/input/marketing-strategy-personalised-offer/train_data.csv
/kaggle/input/marketing-strategy-personalised-offer/test_data.csv


# Loading the data set

In [2]:
train= pd.read_csv("/kaggle/input/marketing-strategy-personalised-offer/train_data.csv")
train

,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food,Offer Accepted
0,2days,₹100000 or More,4~8,1,less1,Married partner,4 star restaurant,36,0,0,...,0,0,4,67,less1,22,Spring,Location B,0,No
1,2days,₹87500 - ₹99999,4~8,0,4~8,Married partner,Take-away restaurant,50plus,0,0,...,0,1,3,89,1~3,18,Summer,Location B,0,Yes
2,2days,₹87500 - ₹99999,less1,1,1~3,Single,Cold drinks,26,1,0,...,1,1,4,67,less1,7,Winter,Location A,1,Yes
3,10hours,₹37500 - ₹49999,less1,0,1~3,Single,Take-away restaurant,46,1,0,...,0,1,3,89,1~3,7,Summer,Location C,0,No
4,2days,₹100000 or More,never,1,1~3,Single,4 star restaurant,21,0,1,...,1,0,3,40,less1,7,Summer,Location C,0,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12374,10hours,₹25000 - ₹37499,less1,1,gt8,Unmarried partner,Restaurant with pub,50plus,0,0,...,1,1,1,89,never,22,Summer,Location B,0,Yes
12375,2days,₹25000 - ₹37499,4~8,0,never,Single,Take-away restaurant,21,0,0,...,1,1,2,67,never,18,Summer,Location B,0,Yes
12376,10hours,₹87500 - ₹99999,4~8,1,gt8,Married partner,Cold drinks,31,0,1,...,1,0,2,89,gt8,18,Spring,Location B,1,No
12377,10hours,₹12500 - ₹24999,4~8,1,less1,Married partner,2 star restaurant,26,0,0,...,1,0,1,89,less1,7,Summer,Location C,0,Yes


In [4]:
train['Offer Accepted'].unique()

array(['No', 'Yes'], dtype=object)

In [3]:
from sklearn.preprocessing import OrdinalEncoder
categories=['₹100000 or More','₹87500 - ₹99999','₹75000 - ₹87499','₹62500 - ₹74999', '₹50000 - ₹62499','₹37500 - ₹49999','₹25000 - ₹37499','₹12500 - ₹24999','Less than ₹12500']
d = {}
val =1
for i in categories:
    d[i]=val
    val+=1
d

{'₹100000 or More': 1,
 '₹87500 - ₹99999': 2,
 '₹75000 - ₹87499': 3,
 '₹62500 - ₹74999': 4,
 '₹50000 - ₹62499': 5,
 '₹37500 - ₹49999': 6,
 '₹25000 - ₹37499': 7,
 '₹12500 - ₹24999': 8,
 'Less than ₹12500': 9}

In [4]:
train['income_range']=train['income_range'].replace(d)
train.isnull().sum()

offer expiration                            0
income_range                                0
no_visited_Cold drinks                    198
travelled_more_than_15mins_for_offer        0
Restaur_spend_less_than20                 121
Marital Status                              0
restaurant type                             0
age                                         0
Prefer western over chinese                 0
travelled_more_than_25mins_for_offer        0
travelled_more_than_5mins_for_offer         0
no_visited_bars                            93
gender                                      0
car                                     12268
restuarant_same_direction_house             0
Cooks regularly                             0
Customer type                               0
Qualification                               0
is foodie                                   0
no_Take-aways                             144
Job/Job Industry                            0
restuarant_opposite_direction_hous

In [5]:
train['temperature'].unique()

array([67, 89, 40])

# Preprocessing

In [6]:
from sklearn.preprocessing import LabelEncoder
train['offer expiration'] = train['offer expiration'].replace({"2days":1,"10hours":2})
train['no_visited_Cold drinks'] = train['no_visited_Cold drinks'].replace({'less1':1,'1~3':2,'4~8':3,'gt8':4,np.NaN:-1,'never':0})
Le = LabelEncoder()
train['Marital Status']=Le.fit_transform(train['Marital Status'])
train['restaurant type']=Le.fit_transform(train['restaurant type'])
train['no_visited_bars'] = train['no_visited_bars'].replace({'less1':1,'1~3':2,'4~8':3,'gt8':4,'never':0})
train['car']=Le.fit_transform(train['car'])
train['no_Take-aways'] = Le.fit_transform(train['no_Take-aways'])
train['Qualification'] = Le.fit_transform(train['Qualification'])
train['Job/Job Industry'] = Le.fit_transform(train['Job/Job Industry'])
train['Restaur_spend_greater_than20'] = train['Restaur_spend_greater_than20'].replace({'less1':1,'1~3':2,'4~8':3,'gt8':4,'never':0})
train['Restaur_spend_less_than20'] = train['Restaur_spend_less_than20'].replace({'less1':1,'1~3':2,'4~8':3,'gt8':4,'never':0})
train['Climate'] = Le.fit_transform(train['Climate'])
train['drop location'] = Le.fit_transform(train['drop location'])
train['Offer Accepted'] = train['Offer Accepted'].replace({'No':0,'Yes':1})
train['age']=train['age'].replace('50plus',50)

train



,offer expiration,income_range,no_visited_Cold drinks,travelled_more_than_15mins_for_offer,Restaur_spend_less_than20,Marital Status,restaurant type,age,Prefer western over chinese,travelled_more_than_25mins_for_offer,...,restuarant_opposite_direction_house,has Children,visit restaurant with rating (avg),temperature,Restaur_spend_greater_than20,Travel Time,Climate,drop location,Prefer home food,Offer Accepted
0,1,1,3,1,1.0,1,1,36,0,0,...,0,0,4,67,1.0,22,0,1,0,0
1,1,2,3,0,3.0,1,4,50,0,0,...,0,1,3,89,2.0,18,1,1,0,1
2,1,2,1,1,2.0,2,2,26,1,0,...,1,1,4,67,1.0,7,2,0,1,1
3,2,6,1,0,2.0,2,4,46,1,0,...,0,1,3,89,2.0,7,1,2,0,0
4,1,1,0,1,2.0,2,1,21,0,1,...,1,0,3,40,1.0,7,1,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12374,2,7,1,1,4.0,3,3,50,0,0,...,1,1,1,89,0.0,22,1,1,0,1
12375,1,7,3,0,0.0,2,4,21,0,0,...,1,1,2,67,0.0,18,1,1,0,1
12376,2,2,3,1,4.0,1,2,31,0,1,...,1,0,2,89,4.0,18,0,1,1,0
12377,2,8,3,1,1.0,1,0,26,0,0,...,1,0,1,89,1.0,7,1,2,0,1


In [7]:
train['age'] = train['age'].replace('below21',21)
train['age']=train['age'].apply(int)
train['gender'] = Le.fit_transform(train['gender'])
train['Customer type'] = Le.fit_transform(train['Customer type'])

In [8]:
train.isna().sum()

offer expiration                          0
income_range                              0
no_visited_Cold drinks                    0
travelled_more_than_15mins_for_offer      0
Restaur_spend_less_than20               121
Marital Status                            0
restaurant type                           0
age                                       0
Prefer western over chinese               0
travelled_more_than_25mins_for_offer      0
travelled_more_than_5mins_for_offer       0
no_visited_bars                          93
gender                                    0
car                                       0
restuarant_same_direction_house           0
Cooks regularly                           0
Customer type                             0
Qualification                             0
is foodie                                 0
no_Take-aways                             0
Job/Job Industry                          0
restuarant_opposite_direction_house       0
has Children                    

# Imputing using knn

In [9]:
from sklearn.impute import KNNImputer
imputer = KNNImputer(n_neighbors=4)
imputer.fit_transform(train)

X = train.drop('Offer Accepted',axis=1)
y = train['Offer Accepted']
# print(X.head())
# print(y.head())
train['Offer Accepted'].unique()
X = imputer.fit_transform(X)
y.shape

(12379,)

# Scaling data 

In [10]:
from sklearn.preprocessing import MinMaxScaler,MaxAbsScaler
# y = StandardScaler().fit_transform(y)
X = MinMaxScaler().fit_transform(X)


# Splitting data into train and test

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit
cv = ShuffleSplit(n_splits=12,random_state = 42)
X_train,X_test,y_train,y_test = train_test_split(X,y,shuffle = cv,random_state=42)


# Creating a baseline model

Score : 0.55

In [14]:
# from sklearn.dummy import DummyClassifier
# dummy = DummyClassifier(random_state = 42,strategy='prior')
# dummy.fit(X_train,y_train)
# dummy.score(X_test,y_test)


# Trying with a different Classifier

# Ridge Classifier

F1 Score: 0.59

In [15]:
# from sklearn.linear_model import RidgeClassifier
# ridge = RidgeClassifier()
# ridge.fit(X_train,y_train)
# print(classification_report(ridge.predict(X_test),y_test))

# Gradient Boosting

F1 Score: 0.62

In [16]:
# from sklearn.ensemble import GradientBoostingClassifier

# GB=GradientBoostingClassifier(random_state=42)
# GB.fit(X_train,y_train)
# y_pred=GB.predict(X_test)
# print(classification_report(y_test,y_pred))



In [17]:
from sklearn.metrics import confusion_matrix
# print(confusion_matrix(y_test,y_pred))

In [18]:
# # from sklearn.ensemble import XGBClassifier
# import xgboost as XGB
# xgb = XGB.XGBClassifier(random_state=42)
# xgb.fit(X_train,y_train)
# ypred=xgb.predict(X_test)
# print(classification_report(y_test,y_pred))

F1 Score: 0.62

In [19]:
 from sklearn.model_selection import GridSearchCV
# param_grid={ 'max_depth': [3,6,10],
#            'learning_rate': [0.01, 0.05, 0.1],
#            }
# clf = GridSearchCV(estimator=GB, 
#                    param_grid=param_grid,
#                    scoring='neg_mean_squared_error', 
#                    verbose=1)
# clf.fit(X_train, y_train)

In [20]:
# y_pred = clf.predict(X_test)
# print(classification_report(y_test,y_pred))

In [21]:
# print("Best parameters:", clf.best_params_)

In [22]:
# best_gb = GradientBoostingClassifier(random_state = 42,learning_rate=0.1,max_depth=3)

# GB With RFE 

F1:Score 0.63

In [23]:
# from sklearn.feature_selection import RFE

# rfe = RFE(GB,n_features_to_select = 23)
# selector = rfe.fit(X_train,y_train)
# print(selector.support_)
# print(selector.ranking_)
# y_pred = rfe.predict(X_test)
# from sklearn.metrics import classification_report
# print(classification_report(y_test,y_pred))

In [24]:
# from sklearn.ensemble import RandomForestClassifier
# rfr = RandomForestClassifier(random_state=42)
# rfr.fit(X_train,y_train)
# ypred=rfr.predict(X_test)
# print(classification_report(y_test,ypred))

In [25]:
from sklearn.model_selection import cross_validate


# XG Boost Algorithm with randomized search cv

In [26]:
from sklearn.model_selection import RandomizedSearchCV
import xgboost as XGB
from sklearn.metrics import classification_report
xga= XGB.XGBClassifier(random_state=42,learning_rate=0.05,max_depth=10,n_estimators=100)
params = { 'max_depth': [3, 5, 6, 10, 15, 20],
           'learning_rate': [0.01, 0.01, 0.02, 0.05],
           'subsample': np.arange(0.5, 1.0, 0.1),
           'colsample_bytree': np.arange(0.4, 1.0, 0.1),
           'colsample_bylevel': np.arange(0.4, 1.0, 0.1),
           'n_estimators': [100, 500, 1000]}
clfa = RandomizedSearchCV(estimator=xga,
                         param_distributions=params,
                         scoring='neg_mean_squared_error',
                         n_iter=25,
                         verbose=1)
clfa.fit(X_train, y_train)
ypred=clfa.predict(X_test)
print(classification_report(y_test,ypred))

Fitting 5 folds for each of 25 candidates, totalling 125 fits
              precision    recall  f1-score   support

           0       0.63      0.44      0.52      1388
           1       0.63      0.79      0.70      1707

    accuracy                           0.63      3095
   macro avg       0.63      0.61      0.61      3095
weighted avg       0.63      0.63      0.62      3095



# Grid Search Cv

In [15]:
# from sklearn.model_selection import GridSearchCV
# import xgboost as XGB
# from sklearn.metrics import classification_report
# xga= XGB.XGBClassifier(random_state=42,learning_rate=0.05,max_depth=10,n_estimators=100)
# params = { 'max_depth': [3, 5, 6, 10, 15, 20],
#            'learning_rate': [0.01, 0.01, 0.02, 0.05],
#            'n_estimators': [100, 500, 1000]}
# clfa = GridSearchCV(estimator=xga,param_grid = params,cv=4)
# clfa.fit(X_train, y_train)
# ypred=clfa.predict(X_test)
# print(classification_report(y_test,ypred))

              precision    recall  f1-score   support

           0       0.61      0.42      0.50      1388
           1       0.62      0.79      0.70      1707

    accuracy                           0.62      3095
   macro avg       0.62      0.60      0.60      3095
weighted avg       0.62      0.62      0.61      3095



In [ ]:
clfa.best_estimator_

# ADABoost Algorithm

f1 score: 0.61

In [ ]:
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.metrics import classification_report
# from sklearn.tree import DecisionTreeClassifier
# param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
#               "base_estimator__splitter" :   ["best", "random"],
#               "n_estimators": [1, 2]
#              }


# DTC = DecisionTreeClassifier(random_state = 11, max_features = "auto", class_weight = "balanced",max_depth = None)

# ada = AdaBoostClassifier(base_estimator = DTC)

# # run grid search
# grid_search_ABC = GridSearchCV(ada, param_grid=param_grid, scoring = 'roc_auc')
# # ada = AdaBoostClassifier(random_state=42,n_estimators=100)
# ada.fit(X_train,y_train)
# y_pred=ada.predict(X_test)
# print(classification_report(y_test,y_pred))

**Best Results XGBoost Classifier with Randomized Search CV**

In [ ]:
# from sklearn.tree import DecisionTreeClassifier
# rfr = DecisionTreeClassifier(random_state=42)
# rfr.fit(X_train,y_train)
# ypred=rfr.predict(X_test)
# print(classification_report(y_test,ypred))

# Working With Test Data

In [ ]:
test = pd.read_csv('/kaggle/input/marketing-strategy-personalised-offer/test_data.csv')
test.head()

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
categories=['₹100000 or More','₹87500 - ₹99999','₹75000 - ₹87499','₹62500 - ₹74999', '₹50000 - ₹62499','₹37500 - ₹49999','₹25000 - ₹37499','₹12500 - ₹24999','Less than ₹12500']
d = {}
val =1
for i in categories:
    d[i]=val
    val+=1
test['income_range']=test['income_range'].replace(d)
from sklearn.preprocessing import LabelEncoder
test['offer expiration'] = test['offer expiration'].replace({"2days":1,"10hours":2})
test['no_visited_Cold drinks'] = test['no_visited_Cold drinks'].replace({'less1':1,'1~3':2,'4~8':3,'gt8':4,np.NaN:-1,'never':0})
Le = LabelEncoder()
test['Marital Status']=Le.fit_transform(test['Marital Status'])
test['restaurant type']=Le.fit_transform(test['restaurant type'])
test['no_visited_bars'] = test['no_visited_bars'].replace({'less1':1,'1~3':2,'4~8':3,'gt8':4,'never':0})
test['car']=Le.fit_transform(test['car'])
test['no_Take-aways'] = Le.fit_transform(test['no_Take-aways'])
test['Qualification'] = Le.fit_transform(test['Qualification'])
test['Job/Job Industry'] = Le.fit_transform(test['Job/Job Industry'])
test['Restaur_spend_greater_than20'] = test['Restaur_spend_greater_than20'].replace({'less1':1,'1~3':2,'4~8':3,'gt8':4,'never':0})
test['Restaur_spend_less_than20'] = test['Restaur_spend_less_than20'].replace({'less1':1,'1~3':2,'4~8':3,'gt8':4,'never':0})
test['Climate'] = Le.fit_transform(test['Climate'])
test['drop location'] = Le.fit_transform(test['drop location'])
# test['Offer Accepted'] = test['Offer Accepted'].replace({'No':0,'Yes':1})
test['age']=test['age'].replace('50plus',50)
test['age'] = test['age'].replace('below21',21)
test['age']=test['age'].apply(int)
test['gender'] = Le.fit_transform(test['gender'])
test['Customer type'] = Le.fit_transform(test['Customer type'])
test['temperature'] =test['temperature'].replace({67:2,89:3,40:1})
test


In [ ]:
imputer = KNNImputer(n_neighbors=4)
imputer.fit_transform(test)

# print(X.head())
# print(y.head())

test = imputer.fit_transform(test)


In [ ]:
from sklearn.preprocessing import MinMaxScaler
# y = StandardScaler().fit_transform(y)
test = MinMaxScaler().fit_transform(test)
test

In [ ]:
# test.isnull.sum()
yhat=clfa.predict(test)

yhat

In [ ]:
dataframe = pd.DataFrame(yhat)
dataframe

In [ ]:
dataframe.columns=["Offer Accepted"]
dataframe.index.names=['id']
dataframe

In [ ]:

dataframe['Offer Accepted']=dataframe['Offer Accepted'].replace({0:"No",1:"Yes"})
dataframe

In [ ]:
dataframe.value_counts()

# Submission File

In [ ]:
dataframe.to_csv('submission.csv')